In [63]:
import pandas as pd
import glob,os
import re
import warnings
warnings.filterwarnings('ignore')

In [64]:
db = pd.read_csv("/Users/damoncrockett/Dropbox/lml/export_07_02_24.csv", encoding='latin-1')

In [65]:
db = db.fillna('')

In [70]:
db.loc[db.Brand.str.contains('Slogas')]

,PhotoID,Year,DateUncertain,Catalog Number,Secondary Catalog Number,Manufacturer,Brand,Format,Texture2,Reflectance2,...,LocationBox,LocationBag,Exposure,BackprintingPrimary,Postcards,Toner,Resin_coated?,ProcessName,Link,AATA_ID
187,1416917868,1916,1,4876,,Illingworth's,Slogas,Package only,[texture unspecified],Glossy,...,101.0,D,,,0,,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
188,-1280115691,1916,1,5130,,Illingworth's,Slogas,Exposed - package open,[texture unspecified],Satin Art,...,114.0,D,,,0,,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0
189,-1257858334,1916,1,5140,,Illingworth's,Slogas,Not exposed - package sealed,[texture unspecified],Glossy,...,114.0,B,,,0,,0,\tgelatin silver process,#http://vocab.getty.edu/page/aat/300139114#,300139114.0


In [4]:
package_enums = [
    1747,2392,2639,4871,4872,4873,4901,5363,5594
]

In [5]:
db['Secondary Catalog Number'].loc[db['Catalog Number'].isin(package_enums)] = db['Secondary Catalog Number'].loc[db['Catalog Number'].isin(package_enums)].apply(lambda x: f'_{x}')

In [6]:
db['catalog'] = db['Catalog Number'].apply(str) + db['Secondary Catalog Number'].apply(lambda x: x.lower())

# exposures

In [7]:
def stringify(v):
    try:
        return str(int(v)).zfill(3)
    except:
        return v

In [8]:
db.Exposure = [stringify(item) for item in db.Exposure]

In [9]:
e = {
    'a': '005',
    'b': '005',
    'c': '006',
    'd': '006',
    'e': '007',
    'f': '007',
    'g': '008',
    'h': '008',
    'i': '010',
    'j': '010',
    'k': '011',
    'l': '012',
    'm': '012',
    'n': '013',
    'o': '013',
    'p': '014',
    'q': '014',
    'r': '015',
    's': '015',
    't': '016',
    'u': '016',
    'v': '017',
    'w': '017',
    'x': '018',
    'y': '019',
    'z': '019',
    'aa': '020',
    'bb': '020',
    'cc': '021',
    'dd': '022',
    'ee': '022',
    'ff': '023',
    'gg': '023',
    'hh': '024',
    'ii': '024',
    'jj': '025',
}

In [10]:
idxs = db.loc[db['Catalog Number']==5500].index

In [11]:
for idx in idxs:
    scn = db['Secondary Catalog Number'].loc[idx]
    db.Exposure.loc[idx] = e[scn]

In [12]:
db.Exposure.loc[db.catalog.str.contains('2563')] = [str(item).zfill(3) for item in range(2,11)]

In [13]:
db.Exposure.loc[db.catalog=='2078t'] = '025'

## Corrections

In [14]:
idx = db.loc[(db['Catalog Number']==2631)&(db['Secondary Catalog Number']=='yy')].iloc[0].name

In [15]:
db['Reflectance2'].loc[idx] = 'Glossy'
db['SurfaceDesignation2'].loc[idx] = 'F'
db.catalog.loc[idx] = '2631xx'

In [16]:
db.catalog.loc[db.catalog=='2081g'] = '2081h'

# img

In [17]:
pkgs = glob.glob(os.path.join("/Users/damoncrockett/paperbase/src/assets/img/packages/","*.jpg"))
sbs = glob.glob(os.path.join("/Users/damoncrockett/paperbase/src/assets/img/samplebooks/","*.jpg"))

In [18]:
pf = pd.DataFrame({"localpath":pkgs})
sf = pd.DataFrame({"localpath":sbs})

# sf

In [19]:
sf['expstem'] = [os.path.basename(item)[:-4] for item in sf.localpath]

In [20]:
sf['catalog_bare'] = [item.split("_")[0] for item in sf.expstem]

In [21]:
sbibarecats = set(sf.catalog_bare)

In [22]:
sbs_without_exposures = ['3194','5440']
for item in sbs_without_exposures:
    sbibarecats.remove(item)

In [23]:
db['sb'] = db['Catalog Number'].apply(str).isin(sbibarecats)

In [24]:
def get_imfile(db,i):
    sb = db.sb.loc[i]
    if not sb:
        return None
    else:
        cn = db['Catalog Number'].loc[i]
        exp = db.Exposure.loc[i]
        return f'{cn}_{exp}.jpg'

In [25]:
db['imfile'] = [get_imfile(db,i) for i in db.index]

In [26]:
from PIL import Image

In [27]:
for imfile in db.imfile:
    if imfile is None:
        continue
    else:
        try:
            localpath = f'/Users/damoncrockett/paperbase/src/assets/img/samplebooks/{imfile}'
            Image.open(localpath)
        except:
            print(imfile)

# pf

In [28]:
pf['fbase'] = [os.path.basename(item) for item in pf.localpath]

In [29]:
pe = set([item.split("_")[0] for item in pf.fbase.loc[pf.fbase.str.contains('_')]])

In [30]:
set([str(item) for item in package_enums]) - pe

set()

In [31]:
pe - set([str(item) for item in package_enums])

{'4870'}

In [32]:
def extract_numbers(s):
    return re.findall(r'\d+', s)

In [33]:
pf['barecat'] = [extract_numbers(item)[0] for item in pf.fbase]

In [34]:
barecats = pf[['barecat','fbase']].groupby('barecat').agg(list).reset_index()

In [35]:
barecats = barecats.loc[~barecats.barecat.isin(pe)]

In [36]:
db.rename(columns={"Catalog Number":"barecat"},inplace=True)

In [37]:
db.barecat = db.barecat.apply(str)

In [38]:
len(db)

7241

In [39]:
db = db.join(barecats.set_index('barecat'),'barecat')

In [40]:
db['imfiles'] = db['imfile'].combine_first(db['fbase'])

In [41]:
db['imfiles_type'] = db.imfiles.apply(type) 

In [42]:
db.imfiles_type.value_counts()

imfiles_type
<class 'str'>      4553
<class 'list'>     2648
<class 'float'>      40
Name: count, dtype: int64

In [43]:
len(db.loc[db.imfile.notnull()])

4553

In [44]:
def tryfile(f):
    if isinstance(f,list):
        for i in f:
            tryfile(i)
    else:
        if not pd.isna(f):
            try:
                localpath = f'/Users/damoncrockett/paperbase/src/assets/img/samplebooks/{f}'
                Image.open(localpath)
            except Exception as e:
                try:
                    localpath = f'/Users/damoncrockett/paperbase/src/assets/img/packages/{f}'
                    Image.open(localpath)
                except Exception as e:
                    print(e)

In [45]:
nullidxs = db.catalog.loc[db.imfiles.isnull()].index

In [46]:
lodashes = pf.loc[pf.barecat.isin(pe)]

In [47]:
lodashes = lodashes[['fbase','barecat']].groupby('barecat').agg(list).reset_index()

In [48]:
for nullidx in nullidxs:
    tmp = db.loc[nullidx]
    catalog = tmp.catalog
    barecat = tmp.barecat

    l = lodashes.fbase.loc[lodashes.barecat==barecat]
    if len(l) > 0:
        assert len(l)==1
        l = l.iloc[0]
        assert isinstance(l,list)

        imglist = []
        core = [item for item in l if "_" not in item]
        if len(core) > 0:
            for c in core:
                imglist.append(c)
        matches = [item for item in l if catalog in item]
        for match in matches:
            imglist.append(match)
    
        if len(imglist) > 0:
            db.imfiles.loc[nullidx] = imglist

In [49]:
for imfile in db.imfiles:
    tryfile(imfile)

In [50]:
misscats = list(db.catalog.loc[db.imfiles.isnull()])

In [51]:
sf.loc[sf.catalog_bare.isin(misscats)]

,localpath,expstem,catalog_bare
2195,/Users/damoncrockett/paperbase/src/assets/img/...,3194_001,3194
2518,/Users/damoncrockett/paperbase/src/assets/img/...,3194_002,3194
4754,/Users/damoncrockett/paperbase/src/assets/img/...,5440_001,5440


In [52]:
idx = db.loc[db.catalog=='5440'].index[0]
db.imfiles.loc[idx] = '5440_001.jpg'

In [53]:
idx = db.loc[db.catalog=='3194'].index[0]
db.imfiles.loc[idx] = ['3194_001.jpg','3194_002.jpg']

In [54]:
db.catalog.loc[db.imfiles.isnull()]

526     1003
2180     114
3167    5617
3168    5618
3169    5619
4125     859
6768    1861
Name: catalog, dtype: object

In [55]:
misscats = list(db.catalog.loc[db.imfiles.isnull()])

In [56]:
pf.loc[pf.barecat.isin(misscats)]

,localpath,fbase,barecat


In [57]:
misscats = [item for item in misscats if item not in ['5617','5618','5619']]

In [58]:
#db.loc[db.catalog.isin(misscats)].iloc[:,1:16]

In [59]:
set(sf.catalog_bare).union(set(pf.barecat)) - set(db.barecat)

{'1005', '5211', '5212', '5262', '5319'}

# checksums

In [60]:
db = db.loc[db.imfiles.notnull()]

In [61]:
db = db.reset_index(drop=True)

In [94]:
import hashlib

In [95]:
def get_md5_checksum(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        # Read and update hash string value in blocks of 4K
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [96]:
db['imfiles_type'] = db.imfiles.apply(type) 

In [102]:
img_dicts = []
for imfile in db.imfiles:
    if isinstance(imfile,str):
        d = {}
        d['filename'] = imfile
        d['md5'] = get_md5_checksum(f'/Users/damoncrockett/lml2lux/img/{imfile}')
        img_dicts.append(d)
    elif isinstance(imfile,list):
        dictlist = []
        for i in imfile:
            d = {}
            d['filename'] = i
            d['md5'] = get_md5_checksum(f'/Users/damoncrockett/lml2lux/img/{i}')
            dictlist.append(d)
        img_dicts.append(dictlist)

In [104]:
db['imfiles_md5'] = img_dicts 

In [107]:
db.to_pickle('db.pkl')